In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Mediapipe 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 손 인식 설정 (두 손 인식)
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# OpenCV 윈도우 설정
cv2.namedWindow('Driving Simulator', cv2.WINDOW_NORMAL)

# 시뮬레이터 변수 초기화
gear = 'P'  # P, N, D
speed = 0
max_speed = 100
acceleration = 50
deceleration = 30

# UI 요소 위치 및 크기
gear_shifter_pos = None
accelerator_pos = None
brake_pos = None
steering_wheel_pos = None
signal_left = False
signal_right = False
signal_left_on = False
signal_right_on = False
wiper_on = False  # 와이퍼 상태
rain_intensity = 0  # 비 강도

# 신호 타이머
signal_timer = 0
signal_interval = 0.5  # 신호 간격 (초)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

def draw_gear_shifter(frame, gear):
    """기어 시프터 그리기"""
    global gear_shifter_pos
    gear_shifter_pos = (frame.shape[1] - 250, 100)  # 위치 조정
    gears = ['P', 'N', 'D']
    shifter_height = 300  # 높이 증가
    
    cv2.rectangle(frame, (gear_shifter_pos[0] - 40, gear_shifter_pos[1]),
                  (gear_shifter_pos[0] + 40, gear_shifter_pos[1] + shifter_height), (255, 255, 255), 2)
    
    for i, g in enumerate(gears):
        y = gear_shifter_pos[1] + i * (shifter_height // 3) + (shifter_height // 6)
        cv2.putText(frame, g, (gear_shifter_pos[0] - 70, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    shifter_pos = gear_shifter_pos[1] + ['P', 'N', 'D'].index(gear) * (shifter_height // 3) + (shifter_height // 6)
    cv2.circle(frame, (gear_shifter_pos[0], shifter_pos), 20, (0, 255, 0), -1)

def draw_pedals(frame, accelerator_pressure, brake_pressure):
    """가속 페달과 브레이크 페달 그리기"""
    global accelerator_pos, brake_pos
    pedal_width = 80
    pedal_height = 160
    
    accelerator_pos = (frame.shape[1] - 220, frame.shape[0] - 200)
    brake_pos = (frame.shape[1] - 120, frame.shape[0] - 200)
    
    # 가속 페달
    cv2.rectangle(frame, accelerator_pos, (accelerator_pos[0] + pedal_width, accelerator_pos[1] + pedal_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (accelerator_pos[0], accelerator_pos[1] + int((1 - accelerator_pressure) * pedal_height)),
                  (accelerator_pos[0] + pedal_width, accelerator_pos[1] + pedal_height), (0, 255, 0), -1)
    cv2.putText(frame, "Accel", (accelerator_pos[0], accelerator_pos[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # 브레이크 페달
    cv2.rectangle(frame, brake_pos, (brake_pos[0] + pedal_width, brake_pos[1] + pedal_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (brake_pos[0], brake_pos[1] + int((1 - brake_pressure) * pedal_height)),
                  (brake_pos[0] + pedal_width, brake_pos[1] + pedal_height), (0, 0, 255), -1)
    cv2.putText(frame, "Brake", (brake_pos[0], brake_pos[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_steering_wheel(frame, angle):
    """스티어링 휠 그리기"""
    global steering_wheel_pos
    steering_wheel_pos = (150, frame.shape[0] - 150)  # 왼쪽 하단으로 위치 조정
    wheel_radius = 150  # 스티어링 휠 크기 증가
    
    # 스티어링 휠 원 그리기
    cv2.circle(frame, steering_wheel_pos, wheel_radius, (255, 255, 255), 2)
    
    # 회전 행렬을 사용하여 핸들을 회전
    angle_rad = np.deg2rad(angle)
    handle_length = wheel_radius
    handle_end = (int(steering_wheel_pos[0] + handle_length * np.cos(angle_rad)),
                  int(steering_wheel_pos[1] - handle_length * np.sin(angle_rad)))
    cv2.line(frame, steering_wheel_pos, handle_end, (0, 255, 0), 2)
    
    # 핸들 표시
    cv2.putText(frame, f'Steering: {angle}°', (steering_wheel_pos[0] - 50, steering_wheel_pos[1] + wheel_radius + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_signals(frame):
    """좌우 신호등 그리기"""
    signal_radius = 20
    signal_spacing = 60
    
    # 왼쪽 신호
    left_signal_pos = (steering_wheel_pos[0] - signal_spacing, steering_wheel_pos[1])
    cv2.circle(frame, left_signal_pos, signal_radius, (0, 255, 255) if signal_left_on and signal_left else (100, 100, 100), -1)
    
    # 오른쪽 신호
    right_signal_pos = (steering_wheel_pos[0] + signal_spacing, steering_wheel_pos[1])
    cv2.circle(frame, right_signal_pos, signal_radius, (0, 255, 255) if signal_right_on and signal_right else (100, 100, 100), -1)

def draw_wiper(frame, wiper_on):
    """와이퍼 그리기"""
    global steering_wheel_pos
    wiper_length = 100
    wiper_width = 10
    wiper_color = (0, 255, 255) if wiper_on else (100, 100, 100)
    
    # 와이퍼 기둥
    wiper_pivot = (steering_wheel_pos[0], steering_wheel_pos[1] - 100)
    cv2.line(frame, wiper_pivot, (wiper_pivot[0], wiper_pivot[1] - wiper_length), wiper_color, wiper_width)
    
    # 와이퍼 팔
    wiper_angle_rad = np.deg2rad(45)
    wiper_end = (int(wiper_pivot[0] + wiper_length * np.cos(wiper_angle_rad)),
                 int(wiper_pivot[1] - wiper_length * np.sin(wiper_angle_rad)))
    cv2.line(frame, wiper_pivot, wiper_end, wiper_color, wiper_width)

def draw_speedometer(frame, speed, max_speed):
    """속도계 그리기"""
    speedometer_pos = (150, 150)  # 속도계의 중심 위치
    speedometer_radius = 100  # 속도계의 반지름
    
    # 속도계 원 그리기
    cv2.circle(frame, speedometer_pos, speedometer_radius, (255, 255, 255), 2)
    
    # 속도계 눈금
    for i in range(0, 101, 10):
        angle_rad = np.deg2rad(180 * (i / 100) - 90)  # 0~100까지를 180도에 매핑
        tick_start = (int(speedometer_pos[0] + (speedometer_radius - 10) * np.cos(angle_rad)),
                      int(speedometer_pos[1] + (speedometer_radius - 10) * np.sin(angle_rad)))
        tick_end = (int(speedometer_pos[0] + speedometer_radius * np.cos(angle_rad)),
                    int(speedometer_pos[1] + speedometer_radius * np.sin(angle_rad)))
        cv2.line(frame, tick_start, tick_end, (255, 255, 255), 1)
    
    # 속도계 바늘 그리기
    speed_angle = np.deg2rad(180 * (speed / max_speed) - 90)
    needle_end = (int(speedometer_pos[0] + (speedometer_radius - 20) * np.cos(speed_angle)),
                  int(speedometer_pos[1] + (speedometer_radius - 20) * np.sin(speed_angle)))
    cv2.line(frame, speedometer_pos, needle_end, (0, 255, 0), 2)
    
    # 속도 숫자 표시
    cv2.putText(frame, f'{int(speed)} km/h', (speedometer_pos[0] - 30, speedometer_pos[1] + speedometer_radius + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

def add_rain_effect(frame, intensity):
    """비 효과 추가"""
    if intensity > 0:
        rain = np.random.rand(frame.shape[0], frame.shape[1]) * 255
        rain = np.uint8(rain)
        rain = cv2.cvtColor(rain, cv2.COLOR_GRAY2BGR)
        rain = cv2.GaussianBlur(rain, (5, 5), 0)
        frame = cv2.addWeighted(frame, 1 - intensity, rain, intensity, 0)
    return frame

def adjust_speed(accelerator_pressure, brake_pressure, current_speed, dt):
    """속도 조정"""
    if accelerator_pressure > 0:
        current_speed += acceleration * accelerator_pressure * dt
    if brake_pressure > 0:
        current_speed -= deceleration * brake_pressure * dt
    return max(0, min(current_speed, max_speed))

# 시뮬레이터 메인 루프
last_time = time.time()
steering_angle = 0
accelerator_pressure = 0
brake_pressure = 0

try:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame or failed to capture image.")
            continue
        
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        
        # 이미지 좌우 반전 및 색상 변환
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe로 손 랜드마크 인식
        result = hands.process(frame_rgb)
        
        left_hand = None
        right_hand = None
        
        if result.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                if handedness.classification[0].label == 'Left':
                    left_hand = hand_landmarks.landmark
                else:
                    right_hand = hand_landmarks.landmark
        
        # 비 효과 추가
        frame = add_rain_effect(frame, rain_intensity)
        
        # UI 그리기
        draw_gear_shifter(frame, gear)
        draw_pedals(frame, accelerator_pressure, brake_pressure)
        draw_steering_wheel(frame, steering_angle)
        draw_signals(frame)
        draw_wiper(frame, wiper_on)  # 와이퍼 그리기
        draw_speedometer(frame, speed, max_speed)  # 속도계 그리기
        
        # 키보드 입력 처리
        key = cv2.waitKey(1) & 0xFF
        if key == ord('u'):  # 가속 페달
            accelerator_pressure = 1
            brake_pressure = 0
        elif key == ord('j'):  # 브레이크 페달
            accelerator_pressure = 0
            brake_pressure = 1 if speed > 0 else 0
        elif key == ord('1'):  # 기어 드라이브
            gear = 'D'
        elif key == ord('2'):  # 기어 중립
            gear = 'N'
        elif key == ord('3'):  # 기어 주차
            gear = 'P'
        elif key == ord('<'):  # 왼쪽 신호
            signal_left = True
            signal_right = False
        elif key == ord('>'):  # 오른쪽 신호
            signal_left = False
            signal_right = True
        elif key == ord(' '):  # 신호 종료
            signal_left = False
            signal_right = False
        elif key == ord('h'):  # 스티어링 휠 시계 방향 회전 (5도 증가)
            steering_angle += 5
        elif key == ord('k'):  # 스티어링 휠 반시계 방향 회전 (5도 감소)
            steering_angle -= 5
        elif key == ord('4'):  # 와이퍼 켜기
            wiper_on = True
        elif key == ord('5'):  # 와이퍼 끄기
            wiper_on = False
        elif key == ord('9'):  # 비 강도 증가
            rain_intensity = min(rain_intensity + 0.1, 1)
        elif key == ord('8'):  # 비 강도 감소
            rain_intensity = max(rain_intensity - 0.1, 0)
        elif key == ord('?'):  # 신호 종료
            signal_left = False
            signal_right = False
        
        # 스티어링 각도 적용 (0~359도로 제한)
        steering_angle = steering_angle % 360
        
        # 속도 조정
        speed = adjust_speed(accelerator_pressure, brake_pressure, speed, dt)
        
        # 신호 타이머 업데이트
        if signal_left or signal_right:
            signal_timer += dt
            if signal_timer >= signal_interval:
                signal_timer = 0
                if signal_left:
                    signal_left_on = not signal_left_on
                if signal_right:
                    signal_right_on = not signal_right_on
        else:
            signal_left_on = False
            signal_right_on = False
        
        # 화면에 프레임 출력
        cv2.imshow('Driving Simulator', frame)
        
        if key == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # 자원 해제
    cap.release()
    cv2.destroyAllWindows()
